In [1]:
!pip install torch torchvision transformers

In [9]:
import os
import json
import time
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
# os: Библиотека для работы с операционной системой, позволяет взаимодействовать с файловой системой.
# json: Библиотека для работы с JSON-файлами.
# time: Библиотека для работы с временем, используется для измерения времени выполнения.
# torch: Основная библиотека для работы с глубоким обучением от PyTorch.
# Dataset, DataLoader: Классы из PyTorch для работы с наборами данных и их загрузкой.
# BertTokenizer, BertModel: Классы из библиотеки transformers, которые используются для обработки текста с помощью модели BERT

class CLEVRDataset(Dataset): # класс CLEVRDataset будет использоваться для загрузки и обработки данных из набора данных CLEVR
    def __init__(self, data_dir, split='train'): # data_dir: путь к директории с данными, split указывает, какую часть данных загружать (по умолчанию 'train')
        self.data_dir = data_dir # сохранение переданного пути к папке с данными (data_dir) в переменной экземпляра self.data_dir
        self.scenes, self.images = self.load_data(split) # вызов метода load_data, передача ему параметр split
        # scenes: это данные о сценах
        # images: это индексы изображений, связанных с этими сценами
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # создание токенизатора, с использованием модели BERT
        # которая называется 'bert-base-uncased'
        # Токенизатор — это инструмент, который разбивает текст на части (токены), чтобы модель могла его понять

    def load_data(self, split): # метод, который загружает данные из файлов
        # split: Это часть данных, которая загружается ('train', 'val' или 'test')
        scenes_path = os.path.join(self.data_dir, 'scenes', f'CLEVR_{split}_scenes.json') # self.data_dir — это папка с данными
        # 'scenes' — это подкаталог, где находятся файлы с данными
        # f'CLEVR_{split}_scenes.json' — это название файла, которое зависит от значения параметра split

        if not os.path.isfile(scenes_path): # проверка существования файла по указанному пути scenes_path
            raise FileNotFoundError(f"Файл не найден: {scenes_path}")

        with open(scenes_path) as f: # используя json.load(f), мы загружаем данные из файла в формате JSON
            scenes_data = json.load(f)['scenes'] # извлечение части данных,  под ключом 'scenes', и сохранение в переменной scenes_data
        
        scenes = [] # scenes: для хранения данных о сценах
        images = [] # images: для хранения индексов изображений, связанных с этими сценами

        for scene in scenes_data: # цикл по каждой сцене в scenes_data
            scenes.append(scene) # добавление в список scenes
            images.append(scene.get('image_index', None)) # # извлечение индекса изображения с помощью scene.get('image_index', None)
            # Если ключ 'image_index' не найден, добавление None

        return scenes, images # возвращение двух списков: scenes и images

    def __len__(self): # определяет, сколько элементов (сцен) содержится в наборе данных
        return len(self.scenes) # возвращает длину списка self.scenes с помощью метода load_data
        # позволит использовать встроенные функции, такие как len(), для получения количества сцен в наборе

    def __getitem__(self, idx): # метод позволяет получать элементы из набора данных по индексу
        # idx: индекс элемента, который нужно получить
        scene = self.scenes[idx] # idx, чтобы получить соответствующую сцену из списка self.scenes
        image_id = self.images[idx] # индекс изображения из списка self.images
        description = self.get_scene_description(scene) # вызов метод get_scene_description, передача ему текущей сцены
        return description, image_id # возвращаем description: текстовое описание сцены, image_id: индекс изображения, связанного с этой сценой

    def get_scene_description(self, scene): # scene:словарь, представляющий сцену, которая содержит информацию о различных объектах в ней
        # метод get для получения списка объектов из сцены
        # ключ 'objects' отсутствует, мы возвращаем пустой список []. Это предотвращает возникновение ошибок, если в сцене нет объектов.
        return " ".join([f"{obj['color']} {obj['shape']}" for obj in scene.get('objects', [])]) # для каждого объекта obj в списке объектов 
        # формируется строка, которая состоит из цвета и формы объекта

class SimpleTransformerModel(torch.nn.Module): # класс SimpleTransformerModel от torch.nn.Module, 
    # позволяет использовать функциональность PyTorch для создания нейронных сетей
    def __init__(self): # __init__ является конструктором класса, который вызывается при создании нового объекта класса
        super(SimpleTransformerModel, self).__init__() 
        self.bert = BertModel.from_pretrained('bert-base-uncased') # загрузка предобученной модели BERT с помощью метода from_pretrained# 
        self.fc = torch.nn.Linear(self.bert.config.hidden_size, 1) # модель bert-base-uncased — это версия BERT, которая не учитывает регистр
# создание линейного слоя (fc), который будет использоваться для преобразования выходных данных BERT в выходное значение
# self.bert.config.hidden_size — это размер скрытого состояния выходного вектора BERT 
# 1 указывает на то, что выходной слой будет возвращать одно значение

    def forward(self, input_ids, attention_mask): # input_ids: тензор, представляющий последовательности токенов, которые будут переданы в модель BERT
    # attention_mask: Это тензор, который используется для указания модели, какие токены должны быть обработаны (1) и какие игнорироваться (0)
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask) # входные данные (input_ids и attention_mask) передаются в предобученную модель BERT
# метод возвращает объект outputs, который содержит различные выходные данные модели, включая скрытые состояния и выход для пула
        pooled_output = outputs.pooler_output # outputs.pooler_output — это выходное значение, полученное после применения слоя пула к выходам BERT
        return self.fc(pooled_output) # после получения pooled_output, он передается в линейный слой self.fc
        # линейный слой преобразует это представление в одно значение, которое будет являться выходом модели

    def summary(self): # 
        
        print("Model Summary:") # выводит заголовок
        print(f"{'Layer':<30} {'Output Shape':<25} {'Param #':<15}") # 
# Форматирует и выводит заголовки для таблицы:
# 'Layer' — название слоя,
# 'Output Shape' — форма выходных данных слоя,
# 'Param #' — количество параметров в слое
        
        print("=" * 70) # выводит горизонтальную линию
        
        for name, param in self.named_parameters(): # Цикл, который перебирает все параметры модели
            
            output_shape = str(param.shape) # получение формы параметра и преобразование ее в строку для вывода
            print(f"{name:<30} {output_shape:<25} {param.numel():<15}") # вывод имя параметра, его форму и количество элементов
        print("=" * 70) # горизонтальная линия
        total_params = sum(p.numel() for p in self.parameters()) # генераторное выражение для подсчета общего количества параметров в модели
        # вызывая метод numel() для каждого параметра
        print(f"Total params: {total_params}") # вывод общего количество параметров в модели

def prepare_data(data_dir, split='train', batch_size=64): # data_dir: директория, в которой находятся данные
    # split: строка, указывающая, какое разбиение данных использовать
    # batch_size: размер пакета, который будет использоваться при загрузке данных
    dataset = CLEVRDataset(data_dir, split) # создание экземпляра класса CLEVRDataset, 
    # который отвечает за загрузку и предобработку данных из указанной директории для данного разбиения
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True) # создается объект DataLoader, который будет использоваться для итерации по датасету
    # batch_size=batch_size указывает размер пакета
    # shuffle=True означает, что данные будут перемешаны перед каждой эпохой, что помогает улучшить обобщающую способность модели
    return dataloader # функция возвращает созданный загрузчик данных

def train_model(model, dataloader, epochs=1): # model: модель, которую нужно обучить
    # dataloader: загрузчик данных, который предоставляет данные для обучения
    # epochs: количество эпох для обучения
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) # оптимизатор Adam с заданной скоростью обучения
    loss_fn = torch.nn.BCEWithLogitsLoss() # функция потерь BCEWithLogitsLoss используется для 
    # бинарной классификации, комбинируя сигмоид и бинарную кросс-энтропию в одном шаге

    model.train() # устанавливает модель в режим обучения, что включает поведение, специфичное для обучения
    total_steps = len(dataloader) # определяет общее количество шагов (батчей) в одной эпохе
    
    for epoch in range(epochs): # цикл по эпохам
        epoch_loss = 0.0 # инициализация переменных для отслеживания потерь и правильных предсказаний
        correct_predictions = 0 
        
        start_time = time.time() # запуск таймера
        
        for step, (descriptions, _) in enumerate(dataloader): # перебирает загрузчик данных, получая описания и метки
            encoding = dataloader.dataset.tokenizer(descriptions, return_tensors='pt', padding=True, truncation=True)
            # токенизирует описания, возвращая тензоры для входных данных

            input_ids = encoding['input_ids'] # получение входных идентификаторов и маски внимания
            attention_mask = encoding['attention_mask'] # 
            labels = torch.zeros(input_ids.size(0), 1)  # создает нулевые метки для обучения

            optimizer.zero_grad() # обнуление градиентов
            outputs = model(input_ids, attention_mask) # прямой проход через модель
            loss = loss_fn(outputs, labels) # вычисление потерь
            loss.backward() # обратное распространение ошибки и шаг оптимизации
            optimizer.step() # 

            epoch_loss += loss.item() # суммирование потерь за эпоху

            if step % 1 == 0: # это условие всегда истинно, так как step % 1 всегда будет равно 0
                # Таким образом, информация о ходе обучения будет выводиться на каждой итерации
                elapsed_time = time.time() - start_time # вычисляется время, прошедшее с начала текущей эпохи, используя time.time()
                print(f"{step + 1}/{total_steps}{elapsed_time:.1f}s - loss: {loss.item():.4f}") 
# Номер текущего шага (step + 1), чтобы начинать с 1 вместо 0
# Общее количество шагов в эпохе (total_steps)
# Время, прошедшее с начала эпохи (elapsed_time), округленное до одного знака после запятой
# Текущую потерю (loss.item()), округленную до четырех знаков после запятой

        avg_loss = epoch_loss / total_steps
# После завершения всех шагов в эпохе, вычисляется средняя потеря, деля общую потерю за эпоху (epoch_loss) на количество шагов (total_steps)
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}") # выводится средняя потеря для текущей эпохи

def predict(model, description):
    model.eval() # модель в режим оценки, что отключает такие механизмы, как дропаут, которые могут влиять на предсказания
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') 
    # токенизатор BERT, который будет использоваться для преобразования текстового описания
    encoding = tokenizer(description, return_tensors='pt', padding=True, truncation=True)
# Токенизирует входное описание, возвращая тензоры для входных идентификаторов и маски внимания
# Параметры padding=True и truncation=True обеспечивают правильное форматирование входных данных. 

    with torch.no_grad(): # для отключения вычисления градиентов
        input_ids = encoding['input_ids'] # получение идентификаторов и маски внимания
        attention_mask = encoding['attention_mask'] 
        output = model(input_ids, attention_mask) # получение предсказания
        
    return output # возврат предсказания

if __name__ == "__main__": # блок кода выполняется только тогда, когда скрипт запускается напрямую, а не импортируется как модуль
    data_dir = "C:/Users/Dasha/Desktop/CLEVR_v1.0" # data_dir: указывает на местоположение данных
    batch_size = 64 # batch_size: определяет количество образцов, обрабатываемых одновременно
    epochs = 1 # epochs: указывает, сколько раз модель будет проходить через весь набор данных

    dataloader = prepare_data(data_dir, split='train', batch_size=batch_size) # prepare_data(...): эта функция загружает и обрабатывает данные, 
# возвращая объект dataloader, который будет использоваться для итерации по данным во время обучения

    model = SimpleTransformerModel() # SimpleTransformerModel(): Создает экземпляр модели

    # Выводим сводку модели до обучения
    model.summary() # model.summary(): функция выводит информацию о структуре модели

    train_model(model, dataloader, epochs=epochs) # train_model(...):функция запускает процесс обучения модели, 
    # используя подготовленные данные и заданное количество эпох

    # Пример предсказания
    # description = "Where is the red cube?"
    # prediction = predict(model, description)
    # print(f"Prediction for description '{description}': {prediction}")


Model Summary:
Layer                          Output Shape              Param #        
bert.embeddings.word_embeddings.weight torch.Size([30522, 768])  23440896       
bert.embeddings.position_embeddings.weight torch.Size([512, 768])    393216         
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])      1536           
bert.embeddings.LayerNorm.weight torch.Size([768])         768            
bert.embeddings.LayerNorm.bias torch.Size([768])         768            
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])    589824         
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])         768            
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])    589824         
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])         768            
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])    589824         
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])